1. Create a conda python3.8 conda env
`conda create --name snowml python=3.8`

2. You need to install these packages locally
 * peft 
 * transformers


In [4]:
!pip install --upgrade --force-reinstall snowflake-snowpark-python==1.8.0

  Using cached snowflake_snowpark_python-1.8.0-py3-none-any.whl (326 kB)
  Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
  Using cached wheel-0.41.3-py3-none-any.whl (65 kB)
  Using cached snowflake_connector_python-3.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.6 MB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (705 kB)
  Using cached cloudpickle-2.0.0-py3-none-any.whl (25 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
  Using cached cffi-1.16.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (443 kB)
  Using cached cryptography-41.0.5-cp37-abi3-manylinux_2_28_x86_64.whl (4.4 MB)
  Using cached oscrypto-1.3.0-py2.py3-none-any.whl (194 kB)
  Using cached pyOpenSSL-23.3.0-py3-none-any.whl (58 kB)
  Using cached pycryptodomex-3.19.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached PyJWT-2.8.0-py3-none-any.whl (22 kB)
  Using cached pytz-2023.3.post1-py2

In [45]:
!pip install --force-reinstall --no-deps /home/ubuntu/snowml/bazel-bin/snowflake/ml/snowflake_ml_python-1.0.12-py3-none-any.whl

Processing /home/ubuntu/snowml/bazel-bin/snowflake/ml/snowflake_ml_python-1.0.12-py3-none-any.whl
  Attempting uninstall: snowflake-ml-python
    Found existing installation: snowflake-ml-python 1.0.12
    Uninstalling snowflake-ml-python-1.0.12:
      Successfully uninstalled snowflake-ml-python-1.0.12


In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2


In [2]:
! pip list | grep snowpark

snowflake-snowpark-python     1.8.0


In [3]:
from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

Connection config available at ~/.snowsql/config

In [4]:
session = Session.builder.configs(SnowflakeLoginOptions('connections.demo')).create()

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


In [5]:
session.get_current_database(), session.get_current_schema()

('"HALU_FT"', '"PUBLIC"')

In [6]:
REGISTRY_DATABASE_NAME = "HALU_MR"
REGISTRY_SCHEMA_NAME = "PUBLIC"

In [7]:
from snowflake.ml.registry import model_registry

model_registry.create_model_registry(
    session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)
registry = model_registry.ModelRegistry(
    session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)

In [47]:
from snowflake.ml.model.models import llm

In [48]:
options = llm.LLMOptions(
    token="...",
    max_batch_size=20,
)
model = llm.LLM(
    model_id_or_path="/home/ubuntu/projects/test_ft_weights",
    options=options
)

In [50]:
svc_model = registry.log_model(
    model_name='build_demo_1101',
    model_version='v5',
    model=model,
    options={"embed_local_ml_library": True},
)

In [51]:
from snowflake.ml.model import deploy_platforms

deployment_options = {
    "compute_pool": 'BUILD_2023_POOL',
    "num_gpus": 1,
    "image_repo": 'sfengineering-servicesnow.registry.snowflakecomputing.com/halu_ft_db/public/haul_repo',
    "enable_remote_image_build": True,
    "model_in_image": True,
    "debug_mode": True,
    #'prebuilt_snowflake_image': 'sfengineering-servicesnow.registry.snowflakecomputing.com/halu_ft_db/public/haul_repo/93d14fc687640746235f8f880a6af8c730ce3eaf:latest'
}
        
deploy_info = svc_model.deploy(
    deployment_name="build_demo_1101_4",
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True,
    options=deployment_options
)
deploy_info

{'name': 'HALU_MR.PUBLIC.build_demo_1101_4',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'infer',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.STRING, name='input')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.STRING, name='generated_text')
                     ]
                 ),
 'options': {'compute_pool': 'BUILD_2023_POOL',
  'num_gpus': 1,
  'image_repo': 'sfengineering-servicesnow.registry.snowflakecomputing.com/halu_ft_db/public/haul_repo',
  'enable_remote_image_build': True,
  'model_in_image': True,
  'debug_mode': True},
 'details': {'image_name': 'sfengineering-servicesnow.registry.snowflakecomputing.com/halu_ft_db/public/haul_repo/50e52d564ecc126d1f53452aa4dd734efa4e3a0a:latest',
  'service_spec': 'spec:\n  container:\n  - env:\n      NUM_WORKERS: 1\n      SNOWML_USE_GPU: true\n   

In [30]:
PROMPT_TEMPLATE = """

[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
Extract JSON response with 'location' and 'toy_list' as keys. Start response by "{".
'location': Location of the caller. Include city only.
'toy_list": List of toy names from the caller.

### Input:
"""

def build_prompt(input):
    return PROMPT_TEMPLATE + input + "\n[/INST]"

In [26]:
import pandas as pd

In [27]:
df = pd.read_json('/home/ubuntu/projects/v8.jsonl', lines=True)

In [36]:
dfl = df['transcript'].to_list()[:10]

In [37]:
prompts = [build_prompt(t) for t in dfl]

In [40]:
input_df = pd.DataFrame({'input': prompts})

In [ ]:
res = svc_model.predict(
    deployment_name='build_demo_1101_2',
    data=input_df
)

In [29]:
input_df = pd.DataFrame({'input': [sample, sample, sample]})

In [30]:
res = svc_model.predict(
    deployment_name='halu_ft_deploy_1',
    data=input_df
)

In [32]:
pd.set_option('display.max_colwidth', None)

In [33]:
res

,generated_text
0,"{""toy_list"": [""Fisher-Price Little People Mickey and Friends""], ""location"": ""Perth""}"
1,"{""toy_list"": [""Fisher-Price Little People Mickey and Friends""], ""location"": ""Perth""}"
2,"{""toy_list"": [""Fisher-Price Little People Mickey and Friends""], ""location"": ""Perth""}"
